In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
test_column_names = ["speedtest100", "speedtest110", "speedtest120", "speedtest130", "speedtest140",
                "speedtest142", "speedtest145", "speedtest160", "speedtest161", "speedtest170", "speedtest180",
                "speedtest190", "speedtest210", "speedtest230", "speedtest240", "speedtest250", "speedtest260",
                "speedtest270", "speedtest280", "speedtest290", "speedtest300", "speedtest320", "speedtest400",
                "speedtest410", "speedtest500", "speedtest510", "speedtest520", "speedtest980", "speedtest990"]
column_names = ["database_type"] + test_column_names

def read_benchmark_data(filename):
    return pd.read_csv(filename, names = column_names)

def filter_by_database_type(data, database_type):
    filtered_by_database_type = data.loc[(data["database_type"] == database_type)]
    #grouped_by_database_type = filtered_by_database_type.groupby('database_type', as_index=False).median()
    return filtered_by_database_type.drop('database_type', 1).reset_index().drop('index', axis=1)

def columns_without_database_type(data):
    return data.values.tolist()[0]

In [ ]:
native_raw_data = read_benchmark_data('benchmark-native-large-pages.csv')
native_memory = filter_by_database_type(native_raw_data, 0)
native_file = filter_by_database_type(native_raw_data, 1)

sgx_raw_data = read_benchmark_data('benchmark-native-sgx.csv')
sgx_memory = filter_by_database_type(sgx_raw_data, 0)
sgx_file = filter_by_database_type(sgx_raw_data, 1)

wasm_raw_data = read_benchmark_data('benchmark-wasm.csv')
wasm_memory = filter_by_database_type(wasm_raw_data, 0)
wasm_file = filter_by_database_type(wasm_raw_data, 1)

wasm_sgx_raw_data = read_benchmark_data('benchmark-wasm-sgx.csv')
wasm_sgx_memory = filter_by_database_type(wasm_sgx_raw_data, 0)
wasm_sgx_file = filter_by_database_type(wasm_sgx_raw_data, 1)

colors = ["#cccccc", "#FB9A99", "#B0DC89", "#33A02C", "#A6CEE3", "#1F78B4", "#FDBF6F", "#FF7F00"]

In [ ]:
# Normalize the results based on native memory implementation
for col in native_memory:
    native_file[col] = native_file[col] / native_memory[col]
    sgx_memory[col] = sgx_memory[col] / native_memory[col]
    sgx_file[col] = sgx_file[col] / native_memory[col]
    wasm_memory[col] = wasm_memory[col] / native_memory[col]
    wasm_file[col] = wasm_file[col] / native_memory[col]
    wasm_sgx_memory[col] = wasm_sgx_memory[col] / native_memory[col]
    wasm_sgx_file[col] = wasm_sgx_file[col] / native_memory[col]
    native_memory[col] = 1

In [ ]:
labels = ['100', '110', '120', '130', '140', '142', '145', '160', '161', '170', '180', '190', '210', '230',
          '240', '250', '260', '270', '280', '290', '300', '320', '400', '410', '500', '510', '520', '980', '990']

x = np.arange(len(labels))  # the label locations
width = 0.7/8.0  # the width of the bars

fig, ax = plt.subplots()
native_memory_bar = ax.bar(x - 7*width/2, native_memory.median().values, width, label='Native (in-memory)')
native_file_bar = ax.bar(x - 5*width/2, native_file.median().values, width, label='Native (in-file)')
sgx_memory_bar = ax.bar(x - 3*width/2, sgx_memory.median().values, width, label='SGX (in-memory)')
sgx_file_bar = ax.bar(x - 1*width/2, sgx_file.median().values, width, label='SGX (in-file)')
wasm_memory_bar = ax.bar(x + 1*width/2, wasm_memory.median().values, width, label='WebAssembly AoT (in-memory)')
wasm_file_bar = ax.bar(x + 3*width/2, wasm_file.median().values, width, label='WebAssembly AoT (in-file)')
wasm_sgx_memory_bar = ax.bar(x + 5*width/2, wasm_sgx_memory.median().values, width, label='WebAssembly AoT in SGX (in-memory)')
wasm_sgx_file_bar = ax.bar(x + 7*width/2, wasm_sgx_file.median().values, width, label='WebAssembly AoT in SGX (in-file)')

i = 0
for bars in [native_memory_bar, native_file_bar, wasm_memory_bar, wasm_file_bar, wasm_sgx_memory_bar, wasm_sgx_file_bar, sgx_memory_bar, sgx_file_bar]:
    for subbar in bars:
        subbar.set_color(colors[i%len(colors)])
    i += 1
        
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Name of experiment')
ax.set_ylabel('Normalised runtime')
ax.set_ylim([0, 15])
ax.set_title('Speedtest1 benchmark')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='upper left')  # reverse to keep order consistent

fig.tight_layout()

plt.show()

In [ ]:
# Export script
#
# Files generated:
# - speedtest1_native_file_formatted.csv
# - speedtest1_wasm_memory_formatted.csv
# - speedtest1_wasm_file_formatted.csv
# - speedtest1_wasm_sgx_memory_formatted.csv
# - speedtest1_wasm_sgx_file_formatted.csv
#
# File format: experiment_name, mean, stddev

def column_name_to_label(column_name):
    return column_name[-3:]

def export_to_file(dataset, filename):
    file = pd.DataFrame(columns = ["experiment_name", "mean", "stddev"])
    
    #dataset = dataset.loc[(dataset["database_type"] == database_type)]
    
    i = 0
    for test_column_name in test_column_names:
        file.loc[i] = [column_name_to_label(test_column_name),
                       dataset[test_column_name].median(),
                       dataset[test_column_name].std()]
        i += 1
        
    display(file)
    file.to_csv(filename, index=False)
    
export_to_file(native_file, 'speedtest1_native_large_pages_file_formatted.csv')
export_to_file(sgx_memory, 'speedtest1_sgx_memory_formatted.csv')
export_to_file(sgx_file, 'speedtest1_sgx_file_formatted.csv')
export_to_file(wasm_memory, 'speedtest1_wasm_memory_formatted.csv')
export_to_file(wasm_file, 'speedtest1_wasm_file_formatted.csv')
export_to_file(wasm_sgx_memory, 'speedtest1_wasm_sgx_memory_formatted.csv')
export_to_file(wasm_sgx_file, 'speedtest1_wasm_sgx_file_formatted.csv')

In [ ]:
wasm_memory

In [ ]:
##
# Stats for the paper.
##

def r_ratio(value):
    return f"{round(value, 1)}"

native_mem_compared_to_wasm_mem = r_ratio((wasm_memory / native_memory).median().median())
print("The slowdown of WAMR memory relative to native is " + native_mem_compared_to_wasm_mem + "x in average")
native_mem_compared_to_wasm_file = r_ratio((wasm_file / native_file).median().median())
print("The slowdown of WAMR file relative to native is " + native_mem_compared_to_wasm_file + "x in average")
print()

wasm_mem_compared_to_sgx_wasm_mem = r_ratio((wasm_sgx_memory / wasm_memory).median().median())
print("The slowdown of Twine memory relative to WAMR is " + wasm_mem_compared_to_sgx_wasm_mem + "x in average")
wasm_mem_compared_to_sgx_wasm_file = r_ratio((wasm_sgx_file / wasm_file).median().median())
print("The slowdown of Twine file relative to WAMR is " + wasm_mem_compared_to_sgx_wasm_file + "x in average")
print()

twine_mem_vs_file_ratio = r_ratio((wasm_sgx_file["speedtest410"] / wasm_sgx_memory["speedtest410"]).median())
print("The slowness of exp. 410 with Twine mem vs file: " + twine_mem_vs_file_ratio + "x")
sgx_mem_vs_file_ratio = r_ratio((sgx_file["speedtest410"] / sgx_memory["speedtest410"]).median())
print("The slowness of exp. 410 with SGX-LKL mem vs file: " + sgx_mem_vs_file_ratio + "x")

In [ ]:
#
## Export to LaTeX
#

f = open("speedtest1-export.tex", "w")

f.write(f"\\def\\speedtestWamrMemToNativeRatio{{{native_mem_compared_to_wasm_mem}}}\n")
f.write(f"\\def\\speedtestWamrFileToNativeRatio{{{native_mem_compared_to_wasm_file}}}\n")
f.write(f"\\def\\speedtestTwineMemToWamrRatio{{{wasm_mem_compared_to_sgx_wasm_mem}}}\n")
f.write(f"\\def\\speedtestTwineFileToWamrRatio{{{wasm_mem_compared_to_sgx_wasm_file}}}\n")
f.write(f"\\def\\speedtestExpFourOneZeroTwineMemVsFileRatio{{{twine_mem_vs_file_ratio}}}\n")
f.write(f"\\def\\speedtestExpFourOneZeroSgxLklMemVsFileRatio{{{sgx_mem_vs_file_ratio}}}\n")


f.close()